In [92]:
import pandas as pd
import numpy as np

In [93]:
sheet = pd.read_csv('./Dropout_Academic Success - Sheet1.csv')
pd.set_option('max_columns', None)

sumDropout = 0
sumGrad = 0
sumEnrolled = 0
for target in sheet.Target.to_numpy():
    if target == 'Dropout':
        sumDropout += 1
    elif target == 'Graduate':
        sumGrad += 1
    elif target == 'Enrolled':
        sumEnrolled += 1

print('Dropout = ' , sumDropout,'   Graduate = ', sumGrad, '       Enrolled = ',sumEnrolled)

sheet['Target'].mask(sheet['Target'] == 'Dropout', 1, inplace = True)
sheet['Target'].mask(sheet['Target'] == 'Graduate', 0, inplace = True)
sheet['Target'].mask(sheet['Target'] == 'Enrolled', 2, inplace = True)

currentStudents = sheet[sheet['Target'] == 2]
data = sheet[sheet.Target != 2]

data

Dropout =  1421    Graduate =  2209        Enrolled =  794


,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,Mother's occupation,Father's occupation,Admission grade,Displaced,Educational special needs,Debtor,Tuition fees up to date,Gender,Scholarship holder,Age at enrollment,International,Curricular units 1st sem (credited),Curricular units 1st sem (enrolled),Curricular units 1st sem (evaluations),Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,1,17,5,171,1,1,122.0,1,19,12,5,9,127.3,1,0,0,1,1,0,20,0,0,0,0,0,0.000000,0,0,0,0,0,0.000000,0,10.8,1.4,1.74,1
1,1,15,1,9254,1,1,160.0,1,1,3,3,3,142.5,1,0,0,0,1,0,19,0,0,6,6,6,14.000000,0,0,6,6,6,13.666667,0,13.9,-0.3,0.79,0
2,1,1,5,9070,1,1,122.0,1,37,37,9,9,124.8,1,0,0,0,1,0,19,0,0,6,0,0,0.000000,0,0,6,0,0,0.000000,0,10.8,1.4,1.74,1
3,1,17,2,9773,1,1,122.0,1,38,37,5,3,119.6,1,0,0,1,0,0,20,0,0,6,8,6,13.428571,0,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,0
4,2,39,1,8014,0,1,100.0,1,37,38,9,9,141.5,0,0,0,1,0,0,45,0,0,6,9,5,12.333333,0,0,6,6,6,13.000000,0,13.9,-0.3,0.79,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4419,1,1,6,9773,1,1,125.0,1,1,1,5,4,122.2,0,0,0,1,1,0,19,0,0,6,7,5,13.600000,0,0,6,8,5,12.666667,0,15.5,2.8,-4.06,0
4420,1,1,2,9773,1,1,120.0,105,1,1,9,9,119.0,1,0,1,0,0,0,18,1,0,6,6,6,12.000000,0,0,6,6,2,11.000000,0,11.1,0.6,2.02,1
4421,1,1,1,9500,1,1,154.0,1,37,37,9,9,149.5,1,0,0,1,0,1,30,0,0,7,8,7,14.912500,0,0,8,9,1,13.500000,0,13.9,-0.3,0.79,1
4422,1,1,1,9147,1,1,180.0,1,37,37,7,4,153.8,1,0,0,1,0,1,20,0,0,5,5,5,13.800000,0,0,5,6,5,12.000000,0,9.4,-0.8,-3.12,0


In [94]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectPercentile

y = data[['Target']].to_numpy().ravel().tolist()
data = data.drop(['Target'], axis = 1)
x = data.to_numpy().tolist()


x = SelectPercentile(percentile=65).fit_transform(x, y)


xTrain, xTest, yTrain, yTest = train_test_split(x, y, train_size = 0.8, random_state=42)


In [95]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
xTrain = scaler.fit_transform(xTrain)
xTest = scaler.transform(xTest)

In [96]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from scipy.stats import loguniform
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold

lrModel = LogisticRegression(penalty = 'none')

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

space = dict()
space['solver'] = ['newton-cg', 'lbfgs', 'liblinear']
space['penalty'] = ['l2']
space['C'] = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]

search = GridSearchCV(lrModel, space, scoring='accuracy', n_jobs=-1, cv=cv)

result = search.fit(xTrain, yTrain)

print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

Best Score: 0.9081786941580757
Best Hyperparameters: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}


In [97]:
optimalModel = LogisticRegression(C = 0.1, penalty = 'l2', solver = 'liblinear')

lrModel.fit(xTrain, yTrain)
ytest_pred = lrModel.predict(xTest)

print('Accuracy on training set:', lrModel.score(xTrain, yTrain))
print('Accuracy on test set:', lrModel.score(xTest, yTest))


print('F1 score on test data: ', f1_score(yTest, ytest_pred))
print('Recall on test data: ', recall_score(yTest, ytest_pred))
print('Precision on test data: ', precision_score(yTest, ytest_pred))

Accuracy on training set: 0.9115013774104683
Accuracy on test set: 0.9077134986225895
F1 score on test data:  0.874296435272045
Recall on test data:  0.8411552346570397
Precision on test data:  0.91015625
